In [ ]:
# 0. Import the necessary libraries
import asyncio
from typing import List, Sequence

from autogen_agentchat.agents import (
    BaseChatAgent,
    CodingAssistantAgent,
    ToolUseAssistantAgent,
)
from autogen_agentchat.agents import AssistantAgent, Handoff
from autogen_agentchat.base import Response
from autogen_agentchat.messages import ChatMessage, StopMessage, TextMessage
from autogen_agentchat.task import Console, TextMentionTermination
from autogen_agentchat.teams import SelectorGroupChat
from autogen_core.base import CancellationToken
from autogen_core.components.tools import FunctionTool
from autogen_ext.models import OpenAIChatCompletionClient
from AzModelClient import create_azure_openai_client


In [ ]:
#1. Create a UserProxyAgent to include the humman in the loop
class UserProxyAgent(BaseChatAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name, "A human user.")

    @property
    def produced_message_types(self) -> List[type[ChatMessage]]:
        return [TextMessage, StopMessage]

    async def on_messages(self, messages: Sequence[ChatMessage], cancellation_token: CancellationToken) -> Response:
        user_input = await asyncio.get_event_loop().run_in_executor(None, input, "Enter your response: ")
        if "TERMINATE" in user_input:
            return Response(chat_message=StopMessage(content="User has terminated the conversation.", source=self.name))
        return Response(chat_message=TextMessage(content=user_input, source=self.name))

    async def on_reset(self, cancellation_token: CancellationToken) -> None:
        pass

In [ ]:
#2. tools for the agents
#2.1 Create a FlightBookingAgent tool to search for flight options.
async def flight_search(start: str, destination: str, date: str) -> str:
    return "\n".join(
        [
            f"AC24 from {start} to {destination} on {date} is $500",
            f"UA23 from {start} to {destination} on {date} is $450",
            f"AL21 from {start} to {destination} on {date} is $400",
        ]
    )

#2.2 Create a FlightBookingAgent tool to book a flight.
async def flight_booking(flight: str, date: str) -> str:
    return f"Booked flight {flight} on {date}"

In [ ]:
#3. The Agent definitions

#3.1 USer proxy agent
user_proxy = UserProxyAgent(name="User")

#3.2 Flight broker agent
flight_broker = AssistantAgent (
    "FlightBroker",
    description="An assistant for booking flights",
    model_client=create_azure_openai_client(),
    tools=[
        FunctionTool(flight_search, description="Search for flights"),
        FunctionTool(flight_booking, description="Book a flight"),
    ],
)

#3.3 Travel assistant agent
travel_assistant = AssistantAgent (
    "TravelAssistant",
    description="A travel assistant",
    model_client=create_azure_openai_client(),
    system_message="You are a travel assistant. If the user confirm not anymore is needed anser with 'TERMINATE'",
)


In [ ]:
#4. Create a team of agents to work together including the humman in the loop
termination = TextMentionTermination("TERMINATE")
team = SelectorGroupChat(
    [user_proxy, flight_broker, travel_assistant],
        #You need the model to analysis the chat and select the right Agent to handle the conversation
    model_client=create_azure_openai_client(),
    termination_condition=termination,
)

In [ ]:
#5. Run the team!!!
stream = team.run_stream(task="Help user plan a trip and book a flight.")
await Console(stream)